In [1]:
import asyncio
from bleak import BleakScanner
from bleak import BleakClient

In [2]:
UART_SERVICE_UUID = "6E400001-B5A3-F393-E0A9-E50E24DCCA9E"
UART_RX_CHAR_UUID = "6E400002-B5A3-F393-E0A9-E50E24DCCA9E"
UART_TX_CHAR_UUID = "6E400003-B5A3-F393-E0A9-E50E24DCCA9E"

start = datetime.datetime.now()

with open("static/db_file.csv","w") as file:
    file.write("# ")
    file.write(start.isoformat())
    file.write("\n")

In [2]:
device = await BleakScanner.find_device_by_name("UART Service")

In [3]:
device

BLEDevice(D77450E0-B097-2719-A036-B77E03F58860, UART Service)

In [3]:
async def uart_server():	
	device = await BleakScanner.find_device_by_name("UART Service")
    
	if device is None:
		print("no UART matching device found")
		sys.exit(1)	

	def handle_disconnect(_: BleakClient):
		print("Device was disconnected, goodbye.")
		for task in asyncio.all_tasks():
			task.cancel()

	def handle_rx(_: BleakGATTCharacteristic, data: bytearray):
		print("received:", data)
		now = datetime.datetime.now()
		rec = str((now-start).seconds) + ","+str(data.decode()).strip()
		with open("static/db_file.csv","a") as file:
			file.write(rec)
			file.write("\n")
			
	async with BleakClient(device, disconnected_callback=handle_disconnect) as client:
		await client.start_notify(UART_TX_CHAR_UUID, handle_rx)
		while True:
			await asyncio.sleep(2)

In [4]:
devices = await discover()

In [5]:
for d in devices:
    if d.name == "UART Service":
        device = d

In [6]:
device

BLEDevice(D77450E0-B097-2719-A036-B77E03F58860, UART Service)

In [ ]:
await read_characteristic(device.address)